In [19]:
import pandas as pd
import numpy as np
import os

In [17]:
data_vulnerabilities = pd.read_json("../smartbugs/dataset/vulnerabilities.json")

In [18]:
data_vulnerabilities.vulnerabilities[0]

[{'lines': [31], 'category': 'access_control'},
 {'lines': [38], 'category': 'access_control'}]

In [20]:
data_vulnerabilities 

,name,path,source,vulnerabilities
0,FibonacciBalance.sol,dataset/access_control/FibonacciBalance.sol,https://github.com/sigp/solidity-security-blog,"[{'lines': [31], 'category': 'access_control'}..."
1,arbitrary_location_write_simple.sol,dataset/access_control/arbitrary_location_writ...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [27], 'category': 'access_control'}]"
2,incorrect_constructor_name1.sol,dataset/access_control/incorrect_constructor_n...,https://github.com/trailofbits/not-so-smart-co...,"[{'lines': [20], 'category': 'access_control'}]"
3,incorrect_constructor_name2.sol,dataset/access_control/incorrect_constructor_n...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [18], 'category': 'access_control'}]"
4,incorrect_constructor_name3.sol,dataset/access_control/incorrect_constructor_n...,https://smartcontractsecurity.github.io/SWC-re...,"[{'lines': [17], 'category': 'access_control'}]"
...,...,...,...,...
138,etherpot_lotto.sol,dataset/unchecked_low_level_calls/etherpot_lot...,https://github.com/etherpot/contract/blob/mast...,"[{'lines': [109], 'category': 'unchecked_low_l..."
139,king_of_the_ether_throne.sol,dataset/unchecked_low_level_calls/king_of_the_...,https://github.com/kieranelby/KingOfTheEtherTh...,"[{'lines': [110], 'category': 'unchecked_low_l..."
140,lotto.sol,dataset/unchecked_low_level_calls/lotto.sol,https://github.com/sigp/solidity-security-blog,"[{'lines': [20], 'category': 'unchecked_low_le..."
141,mishandled.sol,dataset/unchecked_low_level_calls/mishandled.sol,https://github.com/seresistvanandras/EthBench/...,"[{'lines': [14], 'category': 'unchecked_low_le..."


In [62]:
def parse_clean_Data(data,filename):
    indexes_to_drop = []
    for i,d in enumerate(data):
        #Convert bytes to string
        data[i] = data[i].decode("utf-8")
        d = d.decode("utf-8")
        # print(type(d))
        if d.startswith("/*"):
            indexes_to_drop.append(i)
            data[i] = ""
            # pass
        elif d.startswith("*") or d.__contains__("@"):
            indexes_to_drop.append(i)
            data[i] = ""
            # pass
        elif d.startswith(" */"):
            indexes_to_drop.append(i)
            data[i] = ""
            pass
        elif d.startswith("//"):
            indexes_to_drop.append(i)
            data[i] = ""
            pass
        elif d.__contains__("//"):
            data[i]=data[i].split("//")[0]
    # print(indexes_to_drop)
    # result = {f"{filename}":data}
    return data
    # data = data.dropna()
    # data = data.reset_index(drop=True)
    # return data

In [64]:
list_of_dataset=os.listdir("../smartbugs/dataset/")
#Loop to check location is directory or not
final_output = {}
for entry in list_of_dataset:
    # Create full path
    fullpath = os.path.join("../smartbugs/dataset/", entry)
    # If entry is a directory then get the list of files in this directory 
    if os.path.isdir(fullpath):
        print(entry)
        filenames_list = os.listdir(fullpath)
        for filename in filenames_list:
            # print(filename)
            # Read the file content and print it
            with open(os.path.join(fullpath, filename), 'rb') as f:
                file_Data = f.read().splitlines()
                # final_output.append(parse_clean_Data(file_Data, filename))
                final_output[filename] = parse_clean_Data(file_Data, filename)
                # print()
print(final_output)
    

access_control
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 6, 41]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 8, 9, 10, 16, 25, 33, 44]
[0, 1, 2, 3, 5]
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 6]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 11]
[0, 1, 2, 3, 4, 10]
[0, 1, 2, 3, 4, 11]
arithmetic
[0, 1, 2, 3, 4, 8, 9, 10, 11, 38, 39, 40, 41, 42, 50, 51, 52, 53, 60, 61, 62, 76, 77, 78, 85, 86, 87, 88, 97, 98, 100, 101, 102, 103, 110, 111, 112, 113, 128, 134, 135, 144, 145, 146, 147, 154, 155, 156, 158, 167, 176, 185, 186, 196, 197, 198, 199, 208, 216, 224, 232, 240, 241, 243, 276, 277, 279, 280, 294]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3, 4, 6]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3, 4, 6, 7]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4, 6, 7]
[]
[0, 1, 2, 3, 4, 6]
[0, 1, 2, 3

In [61]:
final_output[0]["arbitrary_location_write_simple.sol"]

['',
 '',
 '',
 '',
 '',
 '',
 ' pragma solidity ^0.4.25;',
 '',
 ' contract Wallet {',
 '     uint[] private bonusCodes;',
 '     address private owner;',
 '',
 '     constructor() public {',
 '         bonusCodes = new uint[](0);',
 '         owner = msg.sender;',
 '     }',
 '',
 '     function () public payable {',
 '     }',
 '',
 '     function PushBonusCode(uint c) public {',
 '         bonusCodes.push(c);',
 '     }',
 '',
 '     function PopBonusCode() public {',
 '         ',
 '         require(0 <= bonusCodes.length); ',
 '         bonusCodes.length--; ',
 '     }',
 '',
 '     function UpdateBonusCodeAt(uint idx, uint c) public {',
 '         require(idx < bonusCodes.length);',
 '         bonusCodes[idx] = c; ',
 '     }',
 '',
 '     function Destroy() public {',
 '         require(msg.sender == owner);',
 '         selfdestruct(msg.sender);',
 '     }',
 ' }']

In [65]:
final_output['arbitrary_location_write_simple.sol']

['',
 '',
 '',
 '',
 '',
 '',
 ' pragma solidity ^0.4.25;',
 '',
 ' contract Wallet {',
 '     uint[] private bonusCodes;',
 '     address private owner;',
 '',
 '     constructor() public {',
 '         bonusCodes = new uint[](0);',
 '         owner = msg.sender;',
 '     }',
 '',
 '     function () public payable {',
 '     }',
 '',
 '     function PushBonusCode(uint c) public {',
 '         bonusCodes.push(c);',
 '     }',
 '',
 '     function PopBonusCode() public {',
 '         ',
 '         require(0 <= bonusCodes.length); ',
 '         bonusCodes.length--; ',
 '     }',
 '',
 '     function UpdateBonusCodeAt(uint idx, uint c) public {',
 '         require(idx < bonusCodes.length);',
 '         bonusCodes[idx] = c; ',
 '     }',
 '',
 '     function Destroy() public {',
 '         require(msg.sender == owner);',
 '         selfdestruct(msg.sender);',
 '     }',
 ' }']